### Load stock data

In [0]:
# Load stock data
df = spark.read.parquet("/Volumes/workspace/default/parquetdata/all_stock_data.parquet")

In [0]:
df.count()

34646258

In [0]:
df.columns

['Date',
 'Ticker',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Dividends',
 'Stock Splits']

### Apply transformations

#### Filters

In [0]:
# apply filters early

from pyspark.sql.functions import col, year

# Filter 1: Remove rows with missing or zero closing price
df_filtered = df.filter((col("Close").isNotNull()) & (col("Close") > 0))

# Filter 2: Focus only on data after 2015 for more recent stocks
df_filtered = df_filtered.filter(year(col("Date")) >= 2015)

#### Column transformations

In [0]:
# Add new columns: Daily Return % and Price Range
from pyspark.sql.functions import when, round

df_transformed = (
    df_filtered
    .withColumn(
        "Daily_Return_Percent",
        round(
            when(col("Open") != 0, ((col("Close") - col("Open")) / col("Open")) * 100)
            .otherwise(None),
            2
        )
    )
    .withColumn("Price_Range", round(col("High") - col("Low"), 2))
)

#### Group by aggregations

In [0]:
# group by with aggregation

from pyspark.sql.functions import avg, max as spark_max, sum as spark_sum

# Group by ticker and compute aggregations
agg_df = (
    df_transformed
    .groupBy("Ticker")
    .agg(
        avg("Daily_Return_Percent").alias("Avg_Daily_Return"),
        avg("Volume").alias("Avg_Daily_Volume"),
        spark_max("High").alias("All_Time_High"),
        spark_sum("Dividends").alias("Total_Dividends")
    )
)

#### Slicing, join

In [0]:
# Slice the dataset for select companies

from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Example dataset: sector information
company_info_schema = StructType([
    StructField("Ticker", StringType(), True),
    StructField("Sector", StringType(), True)
])

company_info_data = [
    ("AAPL", "Technology"),
    ("GOOG", "Technology"),
    ("MSFT", "Technology"),
    ("AMZN", "Consumer Discretionary"),
    ("JPM", "Financials")
]

company_info_df = spark.createDataFrame(company_info_data, schema=company_info_schema)

# Join: add sector info to aggregated data
joined_df = agg_df.join(company_info_df, on="Ticker", how="left")

### SQL queries

In [0]:
# sql queries
# Create temp views
df_transformed.createOrReplaceTempView("stocks_daily")
joined_df.createOrReplaceTempView("stocks_summary")

# SQL Query 1: Top 10 highest volatility days
top_volatility = spark.sql("""
    SELECT Ticker, Date, Price_Range
    FROM stocks_daily
    ORDER BY Price_Range ASC
    LIMIT 10
""")

display(top_volatility)

Ticker,Date,Price_Range
HAWLI,2015-01-02,0.0
CARV,2015-01-02,0.0
PDNLA,2015-01-02,0.0
VYBE,2015-01-02,0.0
HAWLL,2015-01-02,0.0
AREN,2015-01-02,0.0
HAWLM,2015-01-02,0.0
NUVI,2015-01-02,0.0
BKSC,2015-01-02,0.0
ASRE,2015-01-02,0.0


In [0]:
# SQL Query 2: Average return and volume by sector
sector_stats = spark.sql("""
    SELECT Sector, 
           ROUND(AVG(Avg_Daily_Return), 2) AS Sector_Avg_Return,
           ROUND(AVG(Avg_Daily_Volume), 0) AS Sector_Avg_Volume
    FROM stocks_summary
    GROUP BY Sector
    ORDER BY Sector_Avg_Return DESC
""")

display(sector_stats)

Sector,Sector_Avg_Return,Sector_Avg_Volume
null,1.06,1033033.0
Technology,0.06,5.9378627E7
Financials,0.03,1.4017142E7
Consumer Discretionary,-0.01,7.6713379E7


In [0]:
# Repartition data by sector to reduce shuffle costs for queries by sector
joined_df = joined_df.repartition("Sector")

#### Write results to destination

In [0]:
output_path = "/Volumes/workspace/default/parquetdata/stock_summary_by_sector/"

joined_df.write.mode("overwrite").partitionBy("Sector").parquet(output_path)

#### Show physical execution plan

In [0]:
#Explain the optimized and physical plan for joined_df with breakdown
joined_df.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (21)
+- == Initial Plan ==
   ColumnarToRow (20)
   +- PhotonResultStage (19)
      +- PhotonShuffleExchangeSource (18)
         +- PhotonShuffleMapStage (17)
            +- PhotonShuffleExchangeSink (16)
               +- PhotonProject (15)
                  +- PhotonBroadcastHashJoin LeftOuter (14)
                     :- PhotonGroupingAgg (7)
                     :  +- PhotonShuffleExchangeSource (6)
                     :     +- PhotonShuffleMapStage (5)
                     :        +- PhotonShuffleExchangeSink (4)
                     :           +- PhotonGroupingAgg (3)
                     :              +- PhotonProject (2)
                     :                 +- PhotonScan parquet  (1)
                     +- PhotonShuffleExchangeSource (13)
                        +- PhotonShuffleMapStage (12)
                           +- PhotonShuffleExchangeSink (11)
                              +- PhotonFilter (10)
                                

#### Caching optimization

In [0]:
import time

# Create a temp view so we can run in-engine SQL
spark.sql("""
    CREATE OR REPLACE TEMP VIEW stock_subset AS
    SELECT *
    FROM workspace.default.all_stock_data
    WHERE Close > 0
    LIMIT 500000
""")

# ACTION 1 (initial uncached read)
start = time.time()
spark.sql("SELECT count(*) FROM stock_subset").show()
print("Uncached read took", round(time.time() - start, 2), "sec")

# Materialize the cache in SQL engine
spark.sql("CACHE TABLE stock_subset")

# ACTION 2 (reads from cache)
start = time.time()
spark.sql("SELECT count(*) FROM stock_subset").show()
print("Cached read took", round(time.time() - start, 2), "sec")

# Optional: clear the cache
spark.sql("UNCACHE TABLE stock_subset")

+--------+
|count(*)|
+--------+
|  500000|
+--------+

Uncached read took 1.13 sec


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8473614661688825>, line 18
     15 print("Uncached read took", round(time.time() - start, 2), "sec")
     17 # Materialize the cache in SQL engine
---> 18 spark.sql("CACHE TABLE stock_subset")
     20 # ACTION 2 (reads from cache)
     21 start = time.time()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:875, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    872         _views.append(SubqueryAlias(df._plan, name))
    874 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 875 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    876 if "sql_command_result" in properties:
    877     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in Sp

### Actions vs Transformations

In [0]:
import time
from pyspark.sql.functions import col

df = spark.table("workspace.default.all_stock_data")

# TRANSFORMATIONS (LAZY)

start = time.time()

# Each of these lines defines *what to do*, but Spark doesn’t run anything yet.
filtered = df.filter(col("Close") > 100)            
projected = filtered.select("Date", "Ticker", "Close", "Volume")  
high_volume = projected.filter(col("Volume") > 1_000_000)        

print(f"Transformations only (lazy): {time.time() - start:.4f}s — no actual computation yet!")

# ACTION 1
print("\nAction 1: count() — triggers Spark job")
start = time.time()
row_count = high_volume.count()
print(f"Count result = {row_count} rows, took {time.time() - start:.2f}s")


# ACTION 2
print("\nAction 2: show() — triggers Spark job again")
start = time.time()
high_volume.show(5)
print(f"show() took {time.time() - start:.2f}s")

print("\n🔍 Notice:")
print(" - Transformations (filter, select) are *lazy* — they only define a plan.")
print(" - Actions (count, show) are *eager* — Spark actually executes the plan each time.")
print(" - Each action triggers its own job, even if they use the same transformations.")

Transformations only (lazy): 16.2245s — no actual computation yet!

Action 1: count() — triggers Spark job
Count result = 354901 rows, took 7.52s

Action 2: show() — triggers Spark job again
+----------+------+------------------+---------+
|      Date|Ticker|             Close|   Volume|
+----------+------+------------------+---------+
|2020-10-14|    BA|163.24000549316406|1.05055E7|
|2020-10-14|  TSCO| 150.5421142578125|1470900.0|
|2020-10-14|   PLD|101.38773345947266|1668000.0|
|2020-10-14|    CI|171.40353393554688|1403700.0|
|2020-10-14|  CDNS|116.97000122070312|1639700.0|
+----------+------+------------------+---------+
only showing top 5 rows
show() took 1.60s

🔍 Notice:
 - Transformations (filter, select) are *lazy* — they only define a plan.
 - Actions (count, show) are *eager* — Spark actually executes the plan each time.
 - Each action triggers its own job, even if they use the same transformations.


### Machine Learning

In [0]:
# combine multiple numerical columns into one feature vector
from pyspark.ml.feature import VectorAssembler

# Define feature input columns
feature_cols = ["Open", "High", "Low", "Volume"]

# Create the assembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# drop missing values
df_ml = (
    df.filter((year(col("Date")) >= 2015))
      .na.drop(subset=["Open", "High", "Low", "Volume", "Close"])  # drop rows with missing values
      .select("Open", "High", "Low", "Volume", "Close")
)

# Transform data
assembled_df = assembler.transform(df_ml).select("features", col("Close").alias("label"))

In [0]:
# split train, test dataset
train_df, test_df = assembled_df.randomSplit([0.8, 0.2], seed=42)

# train regression model
from pyspark.ml.regression import LinearRegression

# Initialize the model
lr = LinearRegression(featuresCol="features", labelCol="label")

# Fit the model
lr_model = lr.fit(train_df)

# Display model coefficients and intercept
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

Coefficients: [0.33333333333333337,0.33333333333333337,0.33333333333333337,0.0003139543871752728]
Intercept: -31520.502386450786


In [0]:
# model evaluation
# Predict on test data
predictions = lr_model.transform(test_df)

# Show some actual vs predicted values
display(predictions.select("label", "prediction").limit(10))

# Evaluate using RMSE and R^2 metrics
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)
rmse = evaluator.evaluate(predictions)
r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2").evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R^2: {r2:.4f}")

label,prediction
-1.1536413768597845E21,-1.1536413768597846E21
-1.1536413768597845E21,-1.1536413768597846E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21
-1.1340880842838812E21,-1.1340880842838813E21


Root Mean Squared Error (RMSE): 31165119.3324
R^2: 1.0000
